In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
from transformers import CLIPProcessor, CLIPModel, AutoTokenizer, AutoModel
import easyocr
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Dropout, Concatenate, BatchNormalization
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

In [3]:
import pandas as pd



CSV_PATH = "/kaggle/input/datasets/akra1234/meme-csv/train_malayalam_ocr (7).csv"

df1 = pd.read_csv(CSV_PATH)
df2=pd.read_excel('/kaggle/input/datasets/akra1234/sth-dravidian/STH_Darvidian_Datasets/mala/Malayalam_Train_label (1).xlsx')
df=pd.merge(df1,df2, on='meme_id', how='inner')
print(df.shape)
df.head()

print(df.shape)
df.head()

(500, 4)
(500, 4)


,meme_id,ml_text_clean,Level 1,Level 2
0,1,1 . ടം ആജ്‌\nര്‍ പ പ ദ്ദ; ടി ഗ്യ ഷ്‌ കാടോ ലാ\n...,TROLL/ OPPOSE,Intersection
1,10,റും ര.\n7 ല്‍ റ ന പ\n൫9൭൭2 ന മി\nടം ന്തര ഞന\n1...,TROLL/ OPPOSE,Against individual person
2,101,വി\n\n4 ിജിിര്തിട\n\n(95 0790൫5 രി:\n\n1) [രിത...,TROLL/ OPPOSE,Against party
3,102,ടട (ധ്യ 7 ഉ ണ്ണ ു\nടം; റ്റി നിം യ; ചി റി ൧\nപം...,TROLL/ OPPOSE,Against individual person
4,104,| ആആ്ഞ്ഞ്ഞ്ഞ്തംത്ത്്തഴ് [|\n)൭(൫70പയല്നുന്ന്ര!...,TROLL/ OPPOSE,Against individual person


In [5]:
def Level1_MAP(label):
    
    if "TROLL/ OPPOSE" in label:
        return 1
    elif "SUPPORT" in label:
        return 0

    else :
        return 'NaN'


def map_level2(label):
    

    if "person" in label:
        return 0
    elif "party" in label:
        return 1
    elif "Intersection" in label:
        return 2
    else:
        return -1

In [6]:
df['label1']=df['Level 1'].map(Level1_MAP)
df['label2']=df["Level 2"].apply(map_level2)

print("Train examples:", len(df))

df.head()

Train examples: 500


,meme_id,ml_text_clean,Level 1,Level 2,label1,label2
0,1,1 . ടം ആജ്‌\nര്‍ പ പ ദ്ദ; ടി ഗ്യ ഷ്‌ കാടോ ലാ\n...,TROLL/ OPPOSE,Intersection,1,2
1,10,റും ര.\n7 ല്‍ റ ന പ\n൫9൭൭2 ന മി\nടം ന്തര ഞന\n1...,TROLL/ OPPOSE,Against individual person,1,0
2,101,വി\n\n4 ിജിിര്തിട\n\n(95 0790൫5 രി:\n\n1) [രിത...,TROLL/ OPPOSE,Against party,1,1
3,102,ടട (ധ്യ 7 ഉ ണ്ണ ു\nടം; റ്റി നിം യ; ചി റി ൧\nപം...,TROLL/ OPPOSE,Against individual person,1,0
4,104,| ആആ്ഞ്ഞ്ഞ്ഞ്തംത്ത്്തഴ് [|\n)൭(൫70പയല്നുന്ന്ര!...,TROLL/ OPPOSE,Against individual person,1,0


In [7]:
df['Level 1'].value_counts()
test_df1=pd.read_csv('/kaggle/input/datasets/akra1234/meme-csv/test_malayalam_ocr (1) (3).csv')
test_df2=pd.read_excel('/kaggle/input/datasets/akra1234/sth-dravidian/STH_Darvidian_Datasets/mala/Malayalam_Test_label.xlsx')
test_df=pd.merge(test_df1, test_df2, on='meme_id', how='inner')
test_df.head()


,meme_id,ml_text_clean,Level 1,Level 2
0,100,1111001000\nകില്‍ ത്ത്‌ ഗള്‍ കം ക ്ന\nറ ഥി വ 4...,NaN,NaN
1,103,) ' 55 ഹ്ന... ടാം പി\nടാ ടം ത്തത്‌\n; ക ത ന ക ...,NaN,NaN
2,106,1.\nവ ്ു വ ി . & യ ക\nലി സ്സ [ പ്ര ി ബ്ല യ നടന...,NaN,NaN
3,107,ന 2011)0200)0/7/11001/01\nയിട്ടു ദിനം.\nയി ാ\n...,NaN,NaN
4,122,യല്ല\nലം ഴു്‌ ലാ ്്‌ ||). [|\n1; റിക ഷ്‌ മി 14...,NaN,NaN


In [9]:
import cv2
import os
import pandas as pd

def load_images_to_dataframe(image_dir):

    data = []


    for file_name in os.listdir(image_dir):
        file_path = os.path.join(image_dir, file_name)
        

        if file_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):

            img = cv2.imread(file_path)
            

            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            

            img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_AREA)
            

            alpha = 1.2 
            beta = 20   
            img = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)


            image_id = int(os.path.splitext(file_name)[0])
            
            # Append image data and file name to the list
            data.append({
                'meme_id': image_id,
                'Image_name': img
            })


    df = pd.DataFrame(data)
    return df

In [10]:
df_test= load_images_to_dataframe('/kaggle/input/datasets/akra1234/sth-dravidian/STH_Darvidian_Datasets/mala/Test_images-20260210T131747Z-1-001/Test_images')
df_train = load_images_to_dataframe('/kaggle/input/datasets/akra1234/sth-dravidian/STH_Darvidian_Datasets/mala/Train_images-20260210T132007Z-1-001/Train_images')
train = pd.merge(df, df_train, on='meme_id')
test = pd.merge(test_df, df_test, on='meme_id')

In [11]:
train.head()

,meme_id,ml_text_clean,Level 1,Level 2,label1,label2,Image_name
0,1,1 . ടം ആജ്‌\nര്‍ പ പ ദ്ദ; ടി ഗ്യ ഷ്‌ കാടോ ലാ\n...,TROLL/ OPPOSE,Intersection,1,2,"[[[20, 20, 20], [20, 20, 20], [20, 20, 20], [2..."
1,10,റും ര.\n7 ല്‍ റ ന പ\n൫9൭൭2 ന മി\nടം ന്തര ഞന\n1...,TROLL/ OPPOSE,Against individual person,1,0,"[[[52, 57, 49], [56, 60, 51], [62, 64, 57], [7..."
2,101,വി\n\n4 ിജിിര്തിട\n\n(95 0790൫5 രി:\n\n1) [രിത...,TROLL/ OPPOSE,Against party,1,1,"[[[28, 28, 28], [28, 28, 28], [28, 28, 28], [2..."
3,102,ടട (ധ്യ 7 ഉ ണ്ണ ു\nടം; റ്റി നിം യ; ചി റി ൧\nപം...,TROLL/ OPPOSE,Against individual person,1,0,"[[[20, 20, 20], [20, 20, 20], [20, 20, 20], [2..."
4,104,| ആആ്ഞ്ഞ്ഞ്ഞ്തംത്ത്്തഴ് [|\n)൭(൫70പയല്നുന്ന്ര!...,TROLL/ OPPOSE,Against individual person,1,0,"[[[55, 55, 55], [55, 55, 55], [55, 55, 55], [5..."


In [12]:
!pip install git+https://github.com/indic-transliteration/indic_transliteration_py/@master

  Cloning https://github.com/indic-transliteration/indic_transliteration_py/ (to revision master) to /tmp/pip-req-build-ncjvavr8
  Running command git clone --filter=blob:none --quiet https://github.com/indic-transliteration/indic_transliteration_py/ /tmp/pip-req-build-ncjvavr8
  Resolved https://github.com/indic-transliteration/indic_transliteration_py/ to commit 322e88533190a874743d72853e3991cbd0e1c076
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done
  Created wheel for indic_transliteration: filename=indic_transliteration-2.3.79-py3-none-any.whl size=164285 sha256=cfa56f54d06b5837bb4d09f8350da4063f30fbc542cda568dd392a20481df1ef
  Stored in directory: /tmp/pip-ephem-wheel-cache-awb5is9b/wheels/ef/dd/4d/c621f9f27e4ec9e07c73a5614faa96a12a4cc1e94bf05f069b
Successfully built indic_transliteration


In [16]:
def text_preprocessing(text):
    import re
    import string
    from indic_transliteration import sanscript
    from indic_transliteration.sanscript import transliterate

    # Remove mentions, hashtags, slashes
    pattern = re.compile(r'[@#/]\S+')
    text = pattern.sub('', text)

    # Remove digits
    text = re.sub(r'\d+', '', text)

    # Remove URLs
    text = re.sub(r'https?://\S+|www\.\S+|ftp://\S+|mailto:\S+', '', text)

    # Remove newline and carriage returns
    text = text.replace('\n', ' ').replace('\r', '')

    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove emojis
    emoji_pattern = re.compile(
        r"["
        r"\U0001F600-\U0001F64F"
        r"\U0001F300-\U0001F5FF"
        r"\U0001F680-\U0001F6FF"
        r"\U0001F1E0-\U0001F1FF"
        r"\U00002700-\U000027BF"
        r"\U0001F900-\U0001F9FF"
        r"\U00002600-\U000026FF"
        r"\U00002B50-\U00002B55"
        r"]+",
        flags=re.UNICODE
    )

    text = emoji_pattern.sub('', text)

    # Malayalam stopwords
    ml_stopwords = [
        "ഒരു","എന്ന്","മറ്റും","ഈ","ഇത്","എന്ന","കൊണ്ട്","എന്നത്","പല","ആണ്",
        "അല്ലെങ്കിൽ","അവൻ","ഞാൻ","ഉള്ള","ആ","ഇവൻ","എന്നാൽ","ആദ്യം","എന്ത്","നിന്ന്",
        "ചില","എന്റെ","പോലെ","വേണ്ടി","വന്ന്","ഇതിന്റെ","അത്","അവൾ","തന്നെ","പലരും",
        "എന്നും","കൂടാതെ","ശേഷം","കொண்ட","ഇരിക്കും","തന്റെ","ഉണ്ട്","സമയം","എപ്പോഴും",
        "അതിന്റെ","തൻ","പിന്നീട്","അവർ","വരെ","നീ","ആയ","ഇരുന്നു","ഉണ്ടായിരുന്നു",
        "വന്ന","ഇരുന്ന","വളരെ","ഇവിടെ","മേൽ","ഇവ","കുറിച്ച്","വരും",
        "മറ്റൊരു","ഇരു","ഇതിൽ","ഇപ്പോൾ","അവന്റെ","മാത്രം","എന്നുള്ള","മുകളിൽ",
        "ചേർന്ന","എനിക്ക്","ഇനിയും","ആ ദിവസം","ഒരേ","വളരെയേറെ","അവിടെ",
        "പലവിധ","വിട്ട്","വലിയ","അதை","കുറിച്ചുള്ള","നിന്റെ","കൂടുതൽ","പേര്","ഇതിനാൽ",
        "അവ","അതേ","എന്തുകൊണ്ട്","രീതി","ആർ","എന്നതിനെ","എല്ലാം","മാത്രമേ",
        "സ്ഥലം","സ്ഥലത്ത്","അതിൽ","നാം","അതിനു","അതുകൊണ്ട്","മറ്റു","ചെറിയ","വിട്ടു","ഏത്",
        "എന്നുവെച്ച്","എന്നറിയപ്പെടുന്ന","എങ്കിലും","അടുത്ത","ഇതിനെ","എടുക്കാൻ","ഇതിന്",
        "ഒഴികെ","പോലും","കുറച്ച്"
    ]

    # Remove stopwords
    words = text.split()
    filtered_words = [word for word in words if word not in ml_stopwords]
    text = " ".join(filtered_words)

    # Transliteration
    text = transliterate(text, sanscript.HK, sanscript.TAMIL)

    return text

In [17]:
train=train.rename(columns={'ml_text_clean': 'text'})
test= test.rename(columns={'ml_text_clean' : 'text'})
train['text'] = train['text'].fillna('').apply(text_preprocessing)
test['text'] = test['text'].fillna('').apply(text_preprocessing)
train['text'] = train['text'].apply(text_preprocessing)
test['text'] = test['text'].apply(text_preprocessing)
train.head()

,meme_id,text,Level 1,Level 2,label1,label2,Image_name
0,1,ടം ആജ്‌ ര്‍ പ പ ദ്ദ ടി ഗ്യ ഷ്‌ കാടോ ലാ പ പ ടം ...,TROLL/ OPPOSE,Intersection,1,2,"[[[20, 20, 20], [20, 20, 20], [20, 20, 20], [2..."
1,10,റും ര ല്‍ റ ന പ ന മി ടം ന്തര ഞന ്‌ പര്‍ ല്‌ ലി...,TROLL/ OPPOSE,Against individual person,1,0,"[[[52, 57, 49], [56, 60, 51], [62, 64, 57], [7..."
2,101,വി ിജിിര്തിട രി രിത്തുതിള്തി സ്‌ ി കസ്യിലാണ്‌ ...,TROLL/ OPPOSE,Against party,1,1,"[[[28, 28, 28], [28, 28, 28], [28, 28, 28], [2..."
3,102,ടട ധ്യ ഉ ണ്ണ ു ടം റ്റി നിം യ ചി റി പം പു ര്‍ ല...,TROLL/ OPPOSE,Against individual person,1,0,"[[[20, 20, 20], [20, 20, 20], [20, 20, 20], [2..."
4,104,ആആ്ഞ്ഞ്ഞ്ഞ്തംത്ത്്തഴ് പയല്നുന്ന്ര ത്ഞ്ത്ൃതന്ത്...,TROLL/ OPPOSE,Against individual person,1,0,"[[[55, 55, 55], [55, 55, 55], [55, 55, 55], [5..."


In [18]:
!pip install deep-translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.5 MB/s eta 0:00:00


In [19]:
print("Distribution for Label1:")
print(train['label1'].value_counts())

# Check counts for label2 (Sub-category)
print("\nDistribution for Label2:")
print(train['label2'].value_counts())

Distribution for Label1:
label1
1    477
0     23
Name: count, dtype: int64

Distribution for Label2:
label2
0    327
1    120
2     53
Name: count, dtype: int64


In [20]:
from PIL import Image
from torchvision import transforms
from deep_translator import GoogleTranslator
import numpy as np
import pandas as pd
import time

augmented_data = []
cnt = 0

# Image augmentation pipeline
img_augmentations = transforms.Compose([
    transforms.ColorJitter(brightness=0.4, contrast=0.3),
    transforms.RandomGrayscale(p=0.1),
    transforms.RandomRotation(10), # Small rotation for variety
])

# Iterate through the Malayalam 'train' dataframe
for idx, row in train.iterrows():
    # Updated Column Names: meme_id, Image_name (pixels), text
    image_data = row['Image_name'] 
    text_content = row['text']
    l1 = row['label1']
    l2 = row['label2']
    meme_id = row['meme_id']

    # --- STRATEGY: Target the extreme minorities ---
    # We augment if Label 1 is 0 OR Label 2 is 2 (Intersection)
    if l1 == 0 or l2 == 2:
        cnt += 1
        if cnt % 5 == 0: print(f"Augmenting Malayalam Minority {cnt}...")

        image_pil = Image.fromarray(np.uint8(image_data))
        
        # We will create 4 variations for each minority row to bridge the gap
        # Languages: English, Tamil, Hindi, Kannada (Dravidian neighbor)
        target_langs = ['en', 'ta', 'hi', 'kn']
        
        for lang in target_langs:
            # 1. Image Transform
            img_aug = np.array(img_augmentations(image_pil))

            # 2. Back-Translation (Malayalam -> Lang -> Malayalam)
            try:
                translated = GoogleTranslator(source='ml', target=lang).translate(text_content)
                time.sleep(0.5) # Safety to avoid 429 Error
                text_aug = GoogleTranslator(source=lang, target='ml').translate(translated)
            except:
                text_aug = text_content # Fallback to original text

            augmented_data.append({
                'meme_id': f"{meme_id}_aug_{lang}",
                'text': text_aug,
                'label1': l1,
                'label2': l2,
                'Image_name': img_aug
            })

# Combine and check
aug_df = pd.DataFrame(augmented_data)
train = pd.concat([train, aug_df], ignore_index=True)

print("\n--- After Augmentation Distribution ---")
print("Label 1:\n", train['label1'].value_counts())
print("Label 2:\n", train['label2'].value_counts())


Augmenting Malayalam Minority 5...
Augmenting Malayalam Minority 10...
Augmenting Malayalam Minority 15...
Augmenting Malayalam Minority 20...
Augmenting Malayalam Minority 25...
Augmenting Malayalam Minority 30...
Augmenting Malayalam Minority 35...
Augmenting Malayalam Minority 40...
Augmenting Malayalam Minority 45...
Augmenting Malayalam Minority 50...
Augmenting Malayalam Minority 55...
Augmenting Malayalam Minority 60...
Augmenting Malayalam Minority 65...
Augmenting Malayalam Minority 70...
Augmenting Malayalam Minority 75...

--- After Augmentation Distribution ---
Label 1:
 label1
1    685
0    115
Name: count, dtype: int64
Label 2:
 label2
0    375
2    265
1    160
Name: count, dtype: int64


In [21]:
# 1. Isolate the "Support" memes that are NOT yet over-augmented
# Focus on those where label2 is 1 or 2 to keep the whole system balanced
target_rows = train[(train['label1'] == 0) & (train['label2'] != 0)].copy()

# 2. If we don't have enough of those, take any label1 == 0
if len(target_rows) < 50:
    target_rows = train[train['label1'] == 0].copy()

more_balanced_data = []
# Use 4 variations per row to add ~400+ samples
final_langs = ['en', 'ta', 'kn', 'te'] 

print(f"Expanding {len(target_rows)} Support memes...")

for idx, row in target_rows.iterrows():
    img_pil = Image.fromarray(np.uint8(row['Image_name'])).convert("RGB")
    
    for lang in final_langs:
        try:
            # Back-translate
            trans = GoogleTranslator(source='ml', target=lang).translate(row['text'])
            time.sleep(0.4)
            text_aug = GoogleTranslator(source=lang, target='ml').translate(trans)
            
            more_balanced_data.append({
                'meme_id': f"{row['meme_id']}_final_{lang}",
                'text': text_aug,
                'label1': 0,
                'label2': row['label2'],
                'Image_name': np.array(img_augmentations(img_pil))
            })
        except:
            continue

# 3. Merge and Check
train = pd.concat([train, pd.DataFrame(more_balanced_data)], ignore_index=True)

print("\n--- FINAL DISTRIBUTION ---")
print("Label 1:\n", train['label1'].value_counts())
print("Label 2:\n", train['label2'].value_counts())


Expanding 55 Support memes...

--- FINAL DISTRIBUTION ---
Label 1:
 label1
1    685
0    335
Name: count, dtype: int64
Label 2:
 label2
0    375
1    360
2    285
Name: count, dtype: int64


In [23]:
# 1. Target a balanced count (around 700 per class)
target_count = 700

# 2. Downsample Label 1 = 0
df_l1_0 = train[train['label1'] == 0]
if len(df_l1_0) > target_count:
    df_l1_0 = df_l1_0.sample(n=target_count, random_state=42)

# 3. Keep all of Label 1 = 1 (since it has 685)
df_l1_1 = train[train['label1'] == 1]

# 4. Combine into a final balanced training set
train_balanced = pd.concat([df_l1_0, df_l1_1]).sample(frac=1, random_state=42).reset_index(drop=True)

print("--- FINAL OPTIMIZED DISTRIBUTION ---")
print(train_balanced['label1'].value_counts())
print(train_balanced['label2'].value_counts())


--- FINAL OPTIMIZED DISTRIBUTION ---
label1
1    685
0    335
Name: count, dtype: int64
label2
0    375
1    360
2    285
Name: count, dtype: int64


In [24]:
print("Distribution for Label1:")
print(train_balanced['label1'].value_counts())
print(train_balanced['label2'].value_counts())


Distribution for Label1:
label1
1    685
0    335
Name: count, dtype: int64
label2
0    375
1    360
2    285
Name: count, dtype: int64


In [25]:
from sklearn.model_selection import train_test_split

# Create a combined key for stratification to keep both labels balanced in the split
stratify_key = train_balanced["label1"].astype(str) + "_" + train_balanced["label2"].astype(str)

# Split the balanced dataframe (85% for training, 15% for validation)
df_train, df_val = train_test_split(
    train_balanced, 
    test_size=0.15, 
    random_state=42, 
    stratify=stratify_key
)

print(f"Training samples: {len(df_train)}")
print(f"Validation samples: {len(df_val)}")


Training samples: 867
Validation samples: 153


In [28]:
train_balanced=train_balanced.dropna()

In [31]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
from transformers import CLIPProcessor, CLIPModel, AutoTokenizer, AutoModel
import easyocr
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Dropout, Concatenate, BatchNormalization
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

2026-02-24 07:02:39.348892: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771916559.512098      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771916559.555602      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771916559.933229      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771916559.933255      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771916559.933258      55 computation_placer.cc:177] computation placer alr

In [40]:
import torch
from transformers import BertTokenizer, BertModel
from transformers import ViTModel, AutoFeatureExtractor  # <- changed here
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from PIL import Image
class TextDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=128):
        self.df = df.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = str(self.df.loc[idx, 'text'])
        l1 = self.df.loc[idx, 'label1']
        l2 = self.df.loc[idx, 'label2']

        encoding = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_len,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'label1': torch.tensor(l1, dtype=torch.long),
            'label2': torch.tensor(l2, dtype=torch.long)
        }

In [43]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

Using device: cuda


In [49]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import torch

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Level 1: Troll vs Support
classes_l1 = np.array([0, 1])  # Must be np.ndarray
y_l1 = df_train['label1'].values
weights_l1 = compute_class_weight(class_weight='balanced', classes=classes_l1, y=y_l1)
weights_l1 = torch.tensor(weights_l1, dtype=torch.float).to(device)

# Level 2: Person / Party / Intersection
classes_l2 = np.array([0, 1, 2])
y_l2 = df_train['label2'].values
weights_l2 = compute_class_weight(class_weight='balanced', classes=classes_l2, y=y_l2)
weights_l2 = torch.tensor(weights_l2, dtype=torch.float).to(device)

# Use in loss
criterion_l1 = nn.CrossEntropyLoss(weight=weights_l1)
criterion_l2 = nn.CrossEntropyLoss(weight=weights_l2)

print("✅ Class weights computed successfully:")
print("Level 1:", weights_l1)
print("Level 2:", weights_l2)

✅ Class weights computed successfully:
Level 1: tensor([1.5264, 0.7436], device='cuda:0')
Level 2: tensor([0.9060, 0.9444, 1.1942], device='cuda:0')


In [52]:
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModel
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Example: Using BERT Multilingual
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
model = AutoModel.from_pretrained('bert-base-multilingual-cased').to(device)

# Minimal Dataset wrapper
class TextDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=128):
        self.df = df.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = str(self.df.loc[idx, 'text'])
        l1 = self.df.loc[idx, 'label1']
        l2 = self.df.loc[idx, 'label2']

        enc = self.tokenizer(
            text, padding='max_length', truncation=True, max_length=self.max_len, return_tensors='pt'
        )

        return {
            'input_ids': enc['input_ids'].squeeze(0),
            'attention_mask': enc['attention_mask'].squeeze(0),
            'label1': torch.tensor(l1, dtype=torch.long),
            'label2': torch.tensor(l2, dtype=torch.long)
        }

# Create DataLoaders
train_dataset = TextDataset(df_train, tokenizer)
dev_dataset = TextDataset(df_val, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=16)

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: bert-base-multilingual-cased
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED |  | 
cls.seq_relationship.weight                | UNEXPECTED |  | 
cls.seq_relationship.bias                  | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED |  | 
cls.predictions.transform.dense.bias       | UNEXPECTED |  | 
cls.predictions.transform.dense.weight     | UNEXPECTED |  | 
cls.predictions.bias                       | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [53]:
import torch.nn as nn
import torch

class TextClassifier(nn.Module):
    def __init__(self, transformer_model, num_l1, num_l2):
        super(TextClassifier, self).__init__()
        self.transformer = transformer_model
        hidden_size = transformer_model.config.hidden_size
        self.dropout = nn.Dropout(0.3)
        self.classifier_l1 = nn.Linear(hidden_size, num_l1)
        self.classifier_l2 = nn.Linear(hidden_size, num_l2)

    def forward(self, input_ids, attention_mask):
        output = self.transformer(input_ids=input_ids, attention_mask=attention_mask)
        # Use [CLS] token
        cls_feat = output.last_hidden_state[:, 0, :]
        cls_feat = self.dropout(cls_feat)
        return self.classifier_l1(cls_feat), self.classifier_l2(cls_feat)

In [54]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = TextClassifier(AutoModel.from_pretrained('bert-base-multilingual-cased'), num_l1=2, num_l2=3).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

epochs = 5  # You can increase

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        ids = batch['input_ids'].to(device)
        mask = batch['attention_mask'].to(device)
        lab1 = batch['label1'].to(device)
        lab2 = batch['label2'].to(device)

        optimizer.zero_grad()
        out1, out2 = model(ids, mask)
        loss = criterion_l1(out1, lab1) + criterion_l2(out2, lab2)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1} | Train Loss: {total_loss/len(train_loader):.4f}")

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: bert-base-multilingual-cased
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED |  | 
cls.seq_relationship.weight                | UNEXPECTED |  | 
cls.seq_relationship.bias                  | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED |  | 
cls.predictions.transform.dense.bias       | UNEXPECTED |  | 
cls.predictions.transform.dense.weight     | UNEXPECTED |  | 
cls.predictions.bias                       | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Epoch 1 | Train Loss: 1.5224
Epoch 2 | Train Loss: 1.2746
Epoch 3 | Train Loss: 1.0790
Epoch 4 | Train Loss: 0.8645
Epoch 5 | Train Loss: 0.9027


In [56]:
from sklearn.metrics import classification_report
val_loader = dev_loader  # just alias it

model.eval()
l1_true, l1_pred, l2_true, l2_pred = [], [], [], []

with torch.no_grad():
    for batch in val_loader:
        ids = batch['input_ids'].to(device)
        mask = batch['attention_mask'].to(device)
        y1 = batch['label1'].to(device)
        y2 = batch['label2'].to(device)

        out1, out2 = model(ids, mask)
        _, p1 = torch.max(out1, 1)
        _, p2 = torch.max(out2, 1)

        l1_true.extend(y1.cpu().numpy())
        l1_pred.extend(p1.cpu().numpy())
        l2_true.extend(y2.cpu().numpy())
        l2_pred.extend(p2.cpu().numpy())

print(classification_report(l1_true, l1_pred, digits=4))
print(classification_report(l2_true, l2_pred, digits=4))

              precision    recall  f1-score   support

           0     0.9787    0.9020    0.9388        51
           1     0.9528    0.9902    0.9712       102

    accuracy                         0.9608       153
   macro avg     0.9658    0.9461    0.9550       153
weighted avg     0.9615    0.9608    0.9604       153

              precision    recall  f1-score   support

           0     0.6923    0.8036    0.7438        56
           1     1.0000    0.7037    0.8261        54
           2     0.7000    0.8140    0.7527        43

    accuracy                         0.7712       153
   macro avg     0.7974    0.7737    0.7742       153
weighted avg     0.8031    0.7712    0.7753       153



In [58]:
import torch
from transformers import AutoTokenizer, AutoModel
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 1️⃣ Tokenizer & Model
xlm_tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
xlm_model = AutoModel.from_pretrained('xlm-roberta-base')

# 2️⃣ Simple Text Classifier for Level 1 & Level 2
class TextClassifier(nn.Module):
    def __init__(self, model, num_l1, num_l2):
        super(TextClassifier, self).__init__()
        self.model = model
        hidden_size = model.config.hidden_size
        self.classifier_l1 = nn.Linear(hidden_size, num_l1)
        self.classifier_l2 = nn.Linear(hidden_size, num_l2)
        self.dropout = nn.Dropout(0.3)

    def forward(self, input_ids, attention_mask):
        out = self.model(input_ids=input_ids, attention_mask=attention_mask)
        # Take the [CLS] token representation
        cls_features = out.last_hidden_state[:, 0, :]
        cls_features = self.dropout(cls_features)
        logits_l1 = self.classifier_l1(cls_features)
        logits_l2 = self.classifier_l2(cls_features)
        return logits_l1, logits_l2

xlm_classifier = TextClassifier(xlm_model, num_l1=2, num_l2=3).to(device)

# 3️⃣ Optional: define a dev_loader for evaluation
# Make sure you have a dataset that uses only text
class TextDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.df = df.reset_index(drop=True)
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = str(self.df.loc[idx, 'text'])
        l1 = self.df.loc[idx, 'label1']
        l2 = self.df.loc[idx, 'label2']
        enc = self.tokenizer(text, padding='max_length', truncation=True, max_length=128, return_tensors='pt')
        return {
            'input_ids': enc['input_ids'].squeeze(0),
            'attention_mask': enc['attention_mask'].squeeze(0),
            'label1': torch.tensor(l1, dtype=torch.long),
            'label2': torch.tensor(l2, dtype=torch.long)
        }

dev_loader = DataLoader(TextDataset(df_val, xlm_tokenizer), batch_size=16)

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

XLMRobertaModel LOAD REPORT from: xlm-roberta-base
Key                       | Status     |  | 
--------------------------+------------+--+-
lm_head.dense.bias        | UNEXPECTED |  | 
lm_head.layer_norm.weight | UNEXPECTED |  | 
lm_head.bias              | UNEXPECTED |  | 
lm_head.layer_norm.bias   | UNEXPECTED |  | 
lm_head.dense.weight      | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [60]:
# ============================
# 1. Imports
# ============================
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd

# ============================
# 2. Device
# ============================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ============================
# 3. Dataset Class
# ============================
class TextDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=128):
        self.df = df.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = str(self.df.loc[idx, 'text'])
        l1 = self.df.loc[idx, 'label1']  # Troll/Support
        l2 = self.df.loc[idx, 'label2']  # Person/Party/Intersection

        encoding = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_len,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'label1': torch.tensor(l1, dtype=torch.long),
            'label2': torch.tensor(l2, dtype=torch.long)
        }

# ============================
# 4. Model Definition
# ============================
class XLMRClassifier(nn.Module):
    def __init__(self, num_classes_l1, num_classes_l2):
        super(XLMRClassifier, self).__init__()
        self.model = AutoModel.from_pretrained('xlm-roberta-base')
        hidden_size = self.model.config.hidden_size  # 768
        self.dropout = nn.Dropout(0.3)
        self.classifier_l1 = nn.Linear(hidden_size, num_classes_l1)
        self.classifier_l2 = nn.Linear(hidden_size, num_classes_l2)

    def forward(self, input_ids, attention_mask):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        # Use [CLS] token (first token)
        pooled_output = outputs.last_hidden_state[:, 0, :]
        pooled_output = self.dropout(pooled_output)
        out1 = self.classifier_l1(pooled_output)
        out2 = self.classifier_l2(pooled_output)
        return out1, out2

# ============================
# 5. Load Data (Replace with your df_train/df_val)
# ============================
# Example format:
# df_train = pd.DataFrame({'text':[...], 'label1':[...], 'label2':[...])
# df_val = pd.DataFrame({'text':[...], 'label1':[...], 'label2':[...])

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')

# Datasets & Loaders
train_dataset = TextDataset(df_train, tokenizer)
val_dataset = TextDataset(df_val, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

# ============================
# 6. Compute Class Weights for LEVEL 1 (Troll/Support)
# ============================
y_train = df_train['label1'].values
classes_l1 = np.array([0, 1])
weights_l1 = compute_class_weight('balanced', classes=classes_l1, y=y_train)
class_weights_l1 = torch.tensor(weights_l1, dtype=torch.float).to(device)
print("Class weights for Level1:", class_weights_l1)

# LEVEL 2 weights optional
y_train_l2 = df_train['label2'].values
classes_l2 = np.array([0,1,2])
weights_l2 = compute_class_weight('balanced', classes=classes_l2, y=y_train_l2)
class_weights_l2 = torch.tensor(weights_l2, dtype=torch.float).to(device)
print("Class weights for Level2:", class_weights_l2)

# ============================
# 7. Initialize Model, Loss, Optimizer
# ============================
num_l1 = 2  # Troll/Support
num_l2 = 3  # Person/Party/Intersection

model = XLMRClassifier(num_l1, num_l2).to(device)
criterion_l1 = nn.CrossEntropyLoss(weight=class_weights_l1)
criterion_l2 = nn.CrossEntropyLoss(weight=class_weights_l2)
optimizer = optim.AdamW(model.parameters(), lr=2e-5)

# ============================
# 8. Training Loop
# ============================
epochs = 3  # adjust as needed

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        ids = batch['input_ids'].to(device)
        mask = batch['attention_mask'].to(device)
        lab1 = batch['label1'].to(device)
        lab2 = batch['label2'].to(device)

        optimizer.zero_grad()
        out1, out2 = model(ids, mask)
        loss = criterion_l1(out1, lab1) + criterion_l2(out2, lab2)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs} | Loss: {total_loss/len(train_loader):.4f}")

# ============================
# 9. Evaluation
# ============================
model.eval()
l1_true, l1_pred = [], []
l2_true, l2_pred = [], []

with torch.no_grad():
    for batch in val_loader:
        ids = batch['input_ids'].to(device)
        mask = batch['attention_mask'].to(device)
        lab1 = batch['label1'].to(device)
        lab2 = batch['label2'].to(device)

        out1, out2 = model(ids, mask)
        _, p1 = torch.max(out1, 1)
        _, p2 = torch.max(out2, 1)

        l1_true.extend(lab1.cpu().numpy())
        l1_pred.extend(p1.cpu().numpy())
        l2_true.extend(lab2.cpu().numpy())
        l2_pred.extend(p2.cpu().numpy())

# ============================
# 10. Print Reports
# ============================
target_names_l1 = ['SUPPORT', 'TROLL/OPPOSE']
target_names_l2 = ['PERSON','PARTY','INTERSECTION']

print("\n🚀 LEVEL 1: Sentiment (Troll vs Support)")
print(classification_report(l1_true, l1_pred, target_names=target_names_l1, digits=4))

print("\n🚀 LEVEL 2: Target (Person/Party/Intersection)")
print(classification_report(l2_true, l2_pred, target_names=target_names_l2, digits=4))

Using device: cuda
Class weights for Level1: tensor([1.5264, 0.7436], device='cuda:0')
Class weights for Level2: tensor([0.9060, 0.9444, 1.1942], device='cuda:0')


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

XLMRobertaModel LOAD REPORT from: xlm-roberta-base
Key                       | Status     |  | 
--------------------------+------------+--+-
lm_head.dense.bias        | UNEXPECTED |  | 
lm_head.layer_norm.weight | UNEXPECTED |  | 
lm_head.bias              | UNEXPECTED |  | 
lm_head.layer_norm.bias   | UNEXPECTED |  | 
lm_head.dense.weight      | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Epoch 1/3 | Loss: 1.6318
Epoch 2/3 | Loss: 1.4903
Epoch 3/3 | Loss: 1.3808

🚀 LEVEL 1: Sentiment (Troll vs Support)
              precision    recall  f1-score   support

     SUPPORT     0.8974    0.6863    0.7778        51
TROLL/OPPOSE     0.8596    0.9608    0.9074       102

    accuracy                         0.8693       153
   macro avg     0.8785    0.8235    0.8426       153
weighted avg     0.8722    0.8693    0.8642       153


🚀 LEVEL 2: Target (Person/Party/Intersection)
              precision    recall  f1-score   support

      PERSON     0.6316    0.8571    0.7273        56
       PARTY     1.0000    0.0741    0.1379        54
INTERSECTION     0.4247    0.7209    0.5345        43

    accuracy                         0.5425       153
   macro avg     0.6854    0.5507    0.4666       153
weighted avg     0.7035    0.5425    0.4651       153



In [61]:
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# -------------------------
# Device
# -------------------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# -------------------------
# Image Transforms
# -------------------------
image_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# -------------------------
# Dataset
# -------------------------
class ImageDataset(Dataset):
    def __init__(self, df, transform):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image_arr = self.df.loc[idx, 'Image_name']
        label1 = self.df.loc[idx, 'label1']
        label2 = self.df.loc[idx, 'label2']

        # Convert to PIL image
        image = Image.fromarray(image_arr.astype('uint8')).convert("RGB")
        image = self.transform(image)

        return {
            'pixel_values': image,
            'label1': torch.tensor(label1, dtype=torch.long),
            'label2': torch.tensor(label2, dtype=torch.long)
        }

# -------------------------
# Model Classes
# -------------------------
class ImageClassifier(nn.Module):
    def __init__(self, base_model_name='resnet50', num_l1=2, num_l2=3):
        super().__init__()
        if base_model_name == 'resnet50':
            self.model = models.resnet50(pretrained=True)
            self.model.fc = nn.Identity()  # remove classifier
            feature_dim = 2048
        elif base_model_name == 'vit':
            from transformers import ViTModel
            self.model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')
            feature_dim = 768
        elif base_model_name == 'clip':
            from transformers import CLIPModel
            self.model = CLIPModel.from_pretrained('openai/clip-vit-base-patch32')
            self.model = self.model.vision_model
            feature_dim = 768
        else:
            raise ValueError("Choose 'resnet50', 'vit', or 'clip'.")

        # Final classifiers
        self.dropout = nn.Dropout(0.3)
        self.classifier_l1 = nn.Linear(feature_dim, num_l1)
        self.classifier_l2 = nn.Linear(feature_dim, num_l2)
        self.base_model_name = base_model_name

    def forward(self, pixel_values):
        if self.base_model_name in ['resnet50']:
            features = self.model(pixel_values)
        else:  # ViT / CLIP
            features = self.model(pixel_values).pooler_output

        features = self.dropout(features)
        return self.classifier_l1(features), self.classifier_l2(features)

# -------------------------
# Dataloaders
# -------------------------
train_loader = DataLoader(ImageDataset(df_train, image_transform), batch_size=16, shuffle=True)
val_loader   = DataLoader(ImageDataset(df_val, image_transform), batch_size=16)

# -------------------------
# Initialize Model
# -------------------------
model_name = 'resnet50'  # change to 'vit' or 'clip'
model = ImageClassifier(base_model_name=model_name, num_l1=2, num_l2=3).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=2e-5)

# -------------------------
# Training Loop
# -------------------------
epochs = 3
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        pix = batch['pixel_values'].to(device)
        lab1 = batch['label1'].to(device)
        lab2 = batch['label2'].to(device)

        optimizer.zero_grad()
        out1, out2 = model(pix)
        loss = criterion(out1, lab1) + criterion(out2, lab2)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs} | Loss: {total_loss/len(train_loader):.4f}")

# -------------------------
# Evaluation
# -------------------------
from sklearn.metrics import classification_report

model.eval()
l1_true, l1_pred = [], []
l2_true, l2_pred = [], []

with torch.no_grad():
    for batch in val_loader:
        pix = batch['pixel_values'].to(device)
        lab1 = batch['label1'].to(device)
        lab2 = batch['label2'].to(device)
        out1, out2 = model(pix)

        _, p1 = torch.max(out1, 1)
        _, p2 = torch.max(out2, 1)

        l1_true.extend(lab1.cpu().numpy())
        l1_pred.extend(p1.cpu().numpy())
        l2_true.extend(lab2.cpu().numpy())
        l2_pred.extend(p2.cpu().numpy())

print("📊 LEVEL 1: Sentiment (Troll vs Support)")
print(classification_report(l1_true, l1_pred, digits=4))
print("\n📊 LEVEL 2: Target (Person/Party/Intersection)")
print(classification_report(l2_true, l2_pred, digits=4))

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 193MB/s] 


Epoch 1/3 | Loss: 1.4285
Epoch 2/3 | Loss: 0.7835
Epoch 3/3 | Loss: 0.4659
📊 LEVEL 1: Sentiment (Troll vs Support)
              precision    recall  f1-score   support

           0     1.0000    0.9216    0.9592        51
           1     0.9623    1.0000    0.9808       102

    accuracy                         0.9739       153
   macro avg     0.9811    0.9608    0.9700       153
weighted avg     0.9748    0.9739    0.9736       153


📊 LEVEL 2: Target (Person/Party/Intersection)
              precision    recall  f1-score   support

           0     0.7500    0.9107    0.8226        56
           1     0.9535    0.7593    0.8454        54
           2     0.9286    0.9070    0.9176        43

    accuracy                         0.8562       153
   macro avg     0.8774    0.8590    0.8619       153
weighted avg     0.8720    0.8562    0.8573       153



In [62]:
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# -------------------------
# Device
# -------------------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# -------------------------
# Image Transforms
# -------------------------
image_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# -------------------------
# Dataset
# -------------------------
class ImageDataset(Dataset):
    def __init__(self, df, transform):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image_arr = self.df.loc[idx, 'Image_name']
        label1 = self.df.loc[idx, 'label1']
        label2 = self.df.loc[idx, 'label2']

        # Convert to PIL image
        image = Image.fromarray(image_arr.astype('uint8')).convert("RGB")
        image = self.transform(image)

        return {
            'pixel_values': image,
            'label1': torch.tensor(label1, dtype=torch.long),
            'label2': torch.tensor(label2, dtype=torch.long)
        }

# -------------------------
# Model Classes
# -------------------------
class ImageClassifier(nn.Module):
    def __init__(self, base_model_name='resnet50', num_l1=2, num_l2=3):
        super().__init__()
        if base_model_name == 'resnet50':
            self.model = models.resnet50(pretrained=True)
            self.model.fc = nn.Identity()  # remove classifier
            feature_dim = 2048
        elif base_model_name == 'vit':
            from transformers import ViTModel
            self.model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')
            feature_dim = 768
        elif base_model_name == 'clip':
            from transformers import CLIPModel
            self.model = CLIPModel.from_pretrained('openai/clip-vit-base-patch32')
            self.model = self.model.vision_model
            feature_dim = 768
        else:
            raise ValueError("Choose 'resnet50', 'vit', or 'clip'.")

        # Final classifiers
        self.dropout = nn.Dropout(0.3)
        self.classifier_l1 = nn.Linear(feature_dim, num_l1)
        self.classifier_l2 = nn.Linear(feature_dim, num_l2)
        self.base_model_name = base_model_name

    def forward(self, pixel_values):
        if self.base_model_name in ['resnet50']:
            features = self.model(pixel_values)
        else:  # ViT / CLIP
            features = self.model(pixel_values).pooler_output

        features = self.dropout(features)
        return self.classifier_l1(features), self.classifier_l2(features)

# -------------------------
# Dataloaders
# -------------------------
train_loader = DataLoader(ImageDataset(df_train, image_transform), batch_size=16, shuffle=True)
val_loader   = DataLoader(ImageDataset(df_val, image_transform), batch_size=16)

# -------------------------
# Initialize Model
# -------------------------
model_name = 'vit'  # change to 'vit' or 'clip'
model = ImageClassifier(base_model_name=model_name, num_l1=2, num_l2=3).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=2e-5)

# -------------------------
# Training Loop
# -------------------------
epochs = 3
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        pix = batch['pixel_values'].to(device)
        lab1 = batch['label1'].to(device)
        lab2 = batch['label2'].to(device)

        optimizer.zero_grad()
        out1, out2 = model(pix)
        loss = criterion(out1, lab1) + criterion(out2, lab2)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs} | Loss: {total_loss/len(train_loader):.4f}")

# -------------------------
# Evaluation
# -------------------------
from sklearn.metrics import classification_report

model.eval()
l1_true, l1_pred = [], []
l2_true, l2_pred = [], []

with torch.no_grad():
    for batch in val_loader:
        pix = batch['pixel_values'].to(device)
        lab1 = batch['label1'].to(device)
        lab2 = batch['label2'].to(device)
        out1, out2 = model(pix)

        _, p1 = torch.max(out1, 1)
        _, p2 = torch.max(out2, 1)

        l1_true.extend(lab1.cpu().numpy())
        l1_pred.extend(p1.cpu().numpy())
        l2_true.extend(lab2.cpu().numpy())
        l2_pred.extend(p2.cpu().numpy())

print("📊 LEVEL 1: Sentiment (Troll vs Support)")
print(classification_report(l1_true, l1_pred, digits=4))
print("\n📊 LEVEL 2: Target (Person/Party/Intersection)")
print(classification_report(l2_true, l2_pred, digits=4))

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/200 [00:00<?, ?it/s]

Epoch 1/3 | Loss: 1.4957
Epoch 2/3 | Loss: 1.0591
Epoch 3/3 | Loss: 0.7031
📊 LEVEL 1: Sentiment (Troll vs Support)
              precision    recall  f1-score   support

           0     1.0000    0.8627    0.9263        51
           1     0.9358    1.0000    0.9668       102

    accuracy                         0.9542       153
   macro avg     0.9679    0.9314    0.9466       153
weighted avg     0.9572    0.9542    0.9533       153


📊 LEVEL 2: Target (Person/Party/Intersection)
              precision    recall  f1-score   support

           0     0.6714    0.8393    0.7460        56
           1     0.9750    0.7222    0.8298        54
           2     0.7442    0.7442    0.7442        43

    accuracy                         0.7712       153
   macro avg     0.7969    0.7686    0.7733       153
weighted avg     0.7990    0.7712    0.7751       153



In [64]:
# ===============================
# 1. Imports
# ===============================
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import pandas as pd
from sklearn.metrics import classification_report

# ===============================
# 2. Device
# ===============================
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ===============================
# 3. Dataset
# ===============================
class ClipImageDataset(Dataset):
    def __init__(self, df, clip_processor):
        self.df = df.reset_index(drop=True)
        self.processor = clip_processor

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_arr = self.df.loc[idx, 'Image_name']  # numpy array
        label1 = self.df.loc[idx, 'label1']
        label2 = self.df.loc[idx, 'label2']

        image = Image.fromarray(img_arr.astype('uint8')).convert("RGB")
        pixel_values = self.processor(images=image, return_tensors="pt")['pixel_values'].squeeze(0)

        return {
            'pixel_values': pixel_values,
            'label1': torch.tensor(label1, dtype=torch.long),
            'label2': torch.tensor(label2, dtype=torch.long)
        }

# ===============================
# 4. CLIP Vision Model
# ===============================
class ClipUnimodal(nn.Module):
    def __init__(self, clip_model, num_l1=2, num_l2=3):
        super(ClipUnimodal, self).__init__()
        self.clip_vision = clip_model.vision_model  # Only use vision
        self.dropout = nn.Dropout(0.3)
        self.classifier_l1 = nn.Linear(768, num_l1)  # CLIP ViT-B/32 -> 768 features
        self.classifier_l2 = nn.Linear(768, num_l2)

    def forward(self, pixel_values):
        features = self.clip_vision(pixel_values=pixel_values).pooler_output
        features = self.dropout(features)
        out1 = self.classifier_l1(features)
        out2 = self.classifier_l2(features)
        return out1, out2

# ===============================
# 5. Load CLIP Processor and Model
# ===============================
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")

# ===============================
# 6. Data Loaders
# ===============================
train_dataset = ClipImageDataset(df_train, clip_processor)
val_dataset = ClipImageDataset(df_val, clip_processor)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

# ===============================
# 7. Initialize Model, Loss, Optimizer
# ===============================
model = ClipUnimodal(clip_model, num_l1=2, num_l2=3).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=2e-5)

# ===============================
# 8. Training Loop
# ===============================
epochs = 3
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        pix = batch['pixel_values'].to(device)
        lab1 = batch['label1'].to(device)
        lab2 = batch['label2'].to(device)

        optimizer.zero_grad()
        out1, out2 = model(pix)

        loss = criterion(out1, lab1) + criterion(out2, lab2)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs} | Loss: {total_loss/len(train_loader):.4f}")

# ===============================
# 9. Evaluation
# ===============================
model.eval()
l1_true, l1_pred = [], []
l2_true, l2_pred = [], []

with torch.no_grad():
    for batch in val_loader:
        pix = batch['pixel_values'].to(device)
        lab1 = batch['label1'].to(device)
        lab2 = batch['label2'].to(device)

        out1, out2 = model(pix)
        _, p1 = torch.max(out1, dim=1)
        _, p2 = torch.max(out2, dim=1)

        l1_true.extend(lab1.cpu().numpy())
        l1_pred.extend(p1.cpu().numpy())
        l2_true.extend(lab2.cpu().numpy())
        l2_pred.extend(p2.cpu().numpy())

# Level 1 Report
target_names_l1 = ['SUPPORT', 'TROLL/OPPOSE']
print("📊 LEVEL 1: Sentiment (Troll vs Support)")
print(classification_report(l1_true, l1_pred, target_names=target_names_l1, digits=4))

# Level 2 Report
target_names_l2 = ['PERSON', 'PARTY', 'INTERSECTION']
print("📊 LEVEL 2: Target (Person/Party/Intersection)")
print(classification_report(l2_true, l2_pred, target_names=target_names_l2, digits=4))

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

The image processor of type `CLIPImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. 


tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

Loading weights:   0%|          | 0/398 [00:00<?, ?it/s]

CLIPModel LOAD REPORT from: openai/clip-vit-base-patch32
Key                                  | Status     |  | 
-------------------------------------+------------+--+-
text_model.embeddings.position_ids   | UNEXPECTED |  | 
vision_model.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Epoch 1/3 | Loss: 0.9923
Epoch 2/3 | Loss: 0.2508
Epoch 3/3 | Loss: 0.0437
📊 LEVEL 1: Sentiment (Troll vs Support)
              precision    recall  f1-score   support

     SUPPORT     1.0000    0.9608    0.9800        51
TROLL/OPPOSE     0.9808    1.0000    0.9903       102

    accuracy                         0.9869       153
   macro avg     0.9904    0.9804    0.9851       153
weighted avg     0.9872    0.9869    0.9869       153

📊 LEVEL 2: Target (Person/Party/Intersection)
              precision    recall  f1-score   support

      PERSON     0.8627    0.7857    0.8224        56
       PARTY     0.8033    0.9074    0.8522        54
INTERSECTION     1.0000    0.9535    0.9762        43

    accuracy                         0.8758       153
   macro avg     0.8887    0.8822    0.8836       153
weighted avg     0.8803    0.8758    0.8761       153

